In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
event_df=pd.read_csv("C:\\Users\\skesi\\Downloads\\Project 2 olympic history\\Project 2 olympic history\\athlete_events.csv")
athlete_df=pd.read_csv("C:\\Users\\skesi\\Downloads\\Project 2 olympic history\\Project 2 olympic history\\athletes.csv")

In [3]:
event_df.head(5)

,athlete_id,games,year,season,city,sport,event,medal
0,1,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [4]:
event_df.isna().sum()

athlete_id         0
games              0
year               0
season             0
city               0
sport              0
event              0
medal         231333
dtype: int64

In [5]:
event_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   athlete_id  271116 non-null  int64 
 1   games       271116 non-null  object
 2   year        271116 non-null  int64 
 3   season      271116 non-null  object
 4   city        271116 non-null  object
 5   sport       271116 non-null  object
 6   event       271116 non-null  object
 7   medal       39783 non-null   object
dtypes: int64(2), object(6)
memory usage: 16.5+ MB


In [6]:
event_df.describe()

,athlete_id,year
count,271116.000000,271116.000000
mean,68248.954396,1978.378480
std,39022.286345,29.877632
min,1.000000,1896.000000
25%,34643.000000,1960.000000
50%,68205.000000,1988.000000
75%,102097.250000,2002.000000
max,135571.000000,2016.000000


In [7]:
athlete_df.head(5)

,id,name,sex,height,weight,team
0,1,A Dijiang,M,180.0,80.0,China
1,2,A Lamusi,M,170.0,60.0,China
2,3,Gunnar Nielsen Aaby,M,NaN,NaN,Denmark
3,4,Edgar Lindenau Aabye,M,NaN,NaN,Denmark/Sweden
4,5,Christine Jacoba Aaftink,F,185.0,82.0,Netherlands


In [8]:
athlete_df.isna().sum()

id            0
name          0
sex           0
height    33916
weight    34885
team          0
dtype: int64

In [9]:
athlete_df['height']=athlete_df['height'].fillna(athlete_df['height'].mean())
athlete_df['weight']=athlete_df['weight'].fillna(athlete_df['weight'].mean())

In [10]:
athlete_df.isna().sum()

id        0
name      0
sex       0
height    0
weight    0
team      0
dtype: int64

In [11]:
athlete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135571 entries, 0 to 135570
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      135571 non-null  int64  
 1   name    135571 non-null  object 
 2   sex     135571 non-null  object 
 3   height  135571 non-null  float64
 4   weight  135571 non-null  float64
 5   team    135571 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 6.2+ MB


In [12]:
athlete_df.describe()

,id,height,weight
count,135571.000000,135571.000000,135571.000000
mean,67786.000000,176.315410,71.961772
std,39136.121009,8.978965,12.550777
min,1.000000,127.000000,25.000000
25%,33893.500000,172.000000,65.000000
50%,67786.000000,176.315410,71.961772
75%,101678.500000,180.000000,76.000000
max,135571.000000,226.000000,214.000000


In [13]:
#  which team has won the maximum gold medals over the years.
# Merge the dataframes
merged_df = pd.merge(event_df, athlete_df, how='inner', left_on='athlete_id', right_on='id')

# Filter for Gold medal winners, group by team, count the number of athletes, and sort
max_gold_team = (
    merged_df[merged_df['medal'] == 'Gold']
    .groupby('team', as_index=False)['athlete_id']
    .count()
    .rename(columns={'athlete_id': 'Count'})
    .sort_values(by='Count', ascending=False)
    .head(1)
)

max_gold_team


,team,Count
242,United States,2515


In [14]:
# for each team print total silver medals and year in which they won maximum silver medal..output 3 columns
# -- team,total_silver_medals, year_of_max_silver
# Total silver medals per team
total_silver_medal = (
    merged_df[merged_df['medal'] == 'Silver']
    .groupby('team', as_index=False)['id'].count()
    .rename(columns={'id': 'Total_silver_medals'})
)

# Year in which each team won the maximum silver medals
year_wise_medals = (
    merged_df[merged_df['medal'] == 'Silver']
    .groupby(['team', 'year'], as_index=False)['id'].count()
    .rename(columns={'id': 'year_silver_medals'})
)

# Find the year with the maximum silver medals for each team
year_of_max_silver = year_wise_medals.loc[
    year_wise_medals.groupby('team')['year_silver_medals'].idxmax()
]

# Merge the total medals and max year DataFrames
result = pd.merge(total_silver_medal, year_of_max_silver, on='team')

# Rename and select required columns
result = result.rename(columns={'year': 'Year_of_max_silver'})[
    ['team', 'Total_silver_medals', 'Year_of_max_silver']
]

# Display the result
result



,team,Total_silver_medals,Year_of_max_silver
0,Albania,1,2000
1,Algeria,4,2016
2,Argentina,84,1996
3,Armenia,5,2016
4,Aschenbrodel,4,1900
...,...,...,...
295,Ylliam VII,1,1968
296,Yugoslavia,180,1988
297,Zambia,1,1996
298,Zimbabwe,4,2008


In [15]:
# --3 which player has won maximum gold medals  amongst the players 
# --which have won only gold medal (never won silver or bronze) over the years

# Players who have never won Silver or Bronze
only_gold_players = merged_df[
    ~merged_df['athlete_id'].isin(
        merged_df[merged_df['medal'].isin(['Silver', 'Bronze'])]['athlete_id']
    )
]

# Among these players, count gold medals and find the player with the maximum
gold_medal_counts = (
    only_gold_players[only_gold_players['medal'] == 'Gold']
    .groupby('name', as_index=False)['id']
    .count()
    .rename(columns={'id': 'Gold_Medals'})
)

# Find the player with the maximum gold medals
max_gold_player = gold_medal_counts.sort_values(by='Gold_Medals', ascending=False).head(1)

max_gold_player


,name,Gold_Medals
5276,"Raymond Clarence ""Ray"" Ewry",10


In [16]:
# --4 in each year which player has won maximum gold medal . Write a query to print year,player name 
# --and no of golds won in that year . In case of a tie print comma separated player names.

player_wise_count=(merged_df[merged_df['medal'] == 'Gold']
                   .groupby(['year','name'],as_index=False)['medal']
                   .count()
                   .rename(columns={'medal':'count'}))
year_wise_max_gold=(player_wise_count
                    .loc[player_wise_count.groupby('year')['count']
                    .idxmax()]
                    .sort_values(by='year')
                    .reset_index(drop=True))
# year_wise_max_gold.drop('index',inplace=True,axis=1)
year_wise_max_gold

,year,name,count
0,1896,Carl Schuhmann,4
1,1900,"Alvin Christian ""Al"" Kraenzlein",4
2,1904,Anton Heida,5
3,1906,Emilio Fontanella,3
4,1908,Henry Taylor,3
5,1912,"Alfred Page ""Al"" Lane",3
6,1920,Nedo Nadi,5
7,1924,Paavo Johannes Nurmi,5
8,1928,"Georg ""Georges"" Miez",3
9,1932,Helene Emma Madison,3


In [17]:
# -5 in which event and year India has won its first gold medal,first silver medal and first bronze medal
# --print 5 columns team,medal,year,name ,sport
ind_df=merged_df[merged_df['team']=='India']
ind_df=ind_df.dropna().reset_index(drop=True)

ind_df=ind_df.loc[ind_df.groupby('medal')['year'].idxmin()]
ind_df[['team','year','medal','name','sport']].reset_index(drop=True)

,team,year,medal,name,sport
0,India,1952,Bronze,Khashaba Dadasaheb Jahdav,Wrestling
1,India,1924,Gold,Antarge Sherpa,Alpinism
2,India,1900,Silver,Norman Gilbert Pritchard (-Trevor),Athletics


In [34]:
# --6 find players who won gold medal in summer and winter olympics both.
# Filter rows where the player won a Gold medal
gold_df = merged_df[merged_df['medal'] == 'Gold']

# Identify Summer and Winter Olympics
summer_olympics = gold_df[gold_df['season'] == 'Summer']
winter_olympics = gold_df[gold_df['season'] == 'Winter']

# Find players who won gold in both Summer and Winter Olympics
players_summer_winter_gold = pd.merge(summer_olympics, 
                                      winter_olympics, 
                                      on='athlete_id', how='inner', suffixes=('_summer', '_winter'))

# Select relevant columns
players_summer_winter_gold = players_summer_winter_gold[['name_summer']].drop_duplicates().rename(columns={'name_summer':'Player_name'}).reset_index(drop=True)

players_summer_winter_gold


,Player_name
0,"Edward Patrick Francis ""Eddie"" Eagan"
1,Gillis Emanuel Grafstrm
